In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer


2024-05-01 06:37:42.947882: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-01 06:37:42.952341: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-01 06:37:43.028497: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 06:37:44.152226: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [70]:
import pandas as pd
import numpy as np

df = pd.read_csv('./ticket_dataset.csv')
df



,Description,Category,Urgency,Sentiment
0,My subscription payment failed.,Issue,Medium,Neutral
1,My subscription payment failed.,Issue,Medium,Neutral
2,The website is slow to load.,Technical Issue,High,Neutral
3,I'm unable to reset my password.,Feature Request,Low,Negative
4,I'm not receiving email notifications from the...,Feature Request,High,Neutral
...,...,...,...,...
995,I'm encountering errors when trying to checkout.,Technical Issue,Medium,Neutral
996,The application crashes whenever I try to open...,Issues,Medium,Neutral
997,I'm experiencing issues with logging into my a...,Technical Issue,Medium,Neutral
998,I'm encountering errors when trying to checkout.,Issues,Medium,Negative


Data Preprocessing

In [71]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer



# Remove special characters and punctuation, and convert to lowercase
df['clean_text'] = df['Description'].apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()))

# Tokenization and removing stopwords
stop_words = set(stopwords.words('english'))
df['clean_text'] = df['clean_text'].apply(lambda x: [word for word in word_tokenize(x) if word not in stop_words])

# Convert list of words back to strings
df['clean_text'] = df['clean_text'].apply(lambda x: ' '.join(x))

df['clean_text'] = df['clean_text'].str.replace('.', '')

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['clean_text'])

df


,Description,Category,Urgency,Sentiment,clean_text
0,My subscription payment failed.,Issue,Medium,Neutral,subscription payment failed
1,My subscription payment failed.,Issue,Medium,Neutral,subscription payment failed
2,The website is slow to load.,Technical Issue,High,Neutral,website slow load
3,I'm unable to reset my password.,Feature Request,Low,Negative,im unable reset password
4,I'm not receiving email notifications from the...,Feature Request,High,Neutral,im receiving email notifications platform
...,...,...,...,...,...
995,I'm encountering errors when trying to checkout.,Technical Issue,Medium,Neutral,im encountering errors trying checkout
996,The application crashes whenever I try to open...,Issues,Medium,Neutral,application crashes whenever try open
997,I'm experiencing issues with logging into my a...,Technical Issue,Medium,Neutral,im experiencing issues logging account
998,I'm encountering errors when trying to checkout.,Issues,Medium,Negative,im encountering errors trying checkout


In [72]:
import spacy
import pandas as pd

# Load the English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

# Apply spaCy to each text in the dataframe
df["doc"] = df["clean_text"].apply(nlp)



# df.to_csv("data/clean_data.csv", index=False)
df


,Description,Category,Urgency,Sentiment,clean_text,doc
0,My subscription payment failed.,Issue,Medium,Neutral,subscription payment failed,"(subscription, payment, failed)"
1,My subscription payment failed.,Issue,Medium,Neutral,subscription payment failed,"(subscription, payment, failed)"
2,The website is slow to load.,Technical Issue,High,Neutral,website slow load,"(website, slow, load)"
3,I'm unable to reset my password.,Feature Request,Low,Negative,im unable reset password,"(i, m, unable, reset, password)"
4,I'm not receiving email notifications from the...,Feature Request,High,Neutral,im receiving email notifications platform,"(i, m, receiving, email, notifications, platform)"
...,...,...,...,...,...,...
995,I'm encountering errors when trying to checkout.,Technical Issue,Medium,Neutral,im encountering errors trying checkout,"(i, m, encountering, errors, trying, checkout)"
996,The application crashes whenever I try to open...,Issues,Medium,Neutral,application crashes whenever try open,"(application, crashes, whenever, try, open)"
997,I'm experiencing issues with logging into my a...,Technical Issue,Medium,Neutral,im experiencing issues logging account,"(i, m, experiencing, issues, logging, account)"
998,I'm encountering errors when trying to checkout.,Issues,Medium,Negative,im encountering errors trying checkout,"(i, m, encountering, errors, trying, checkout)"


In [75]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

# Load the pre-trained sentiment analysis model from TensorFlow Hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
model = hub.load(module_url)


# Process text data
texts = df['Description']

# Apply sentiment analysis model
embeddings = model(texts)
sentiment_scores = tf.reduce_mean(embeddings, axis=1).numpy()*100
df["sentiment_scores" ] = sentiment_scores
# Define a function to categorize sentiment
def categorize_sentiment(score):
    if score > 0.6:
        return 'Positive'
    elif score < 0.2:
        return 'Negative'
    else:
        return 'Neutral'

# Apply the function to each sentiment score
df['sentiment'] = sentiment_scores
df['sentiment'] = df['sentiment'].apply(categorize_sentiment)

# Define a function to determine urgency class based on sentiment
def determine_urgency(sentiment):
    if sentiment == "Positive":
        return "Low"
    elif sentiment == "Negative":
        return "High"
    else:
        return "Medium"

# Apply the function to each sentiment
df["urgency_class"] = df["sentiment"].apply(determine_urgency)

df


In [ ]:
def extract_features(text):
  """
  This function extracts features from the preprocessed text.

  Args:
      text: A list of tokens (words or phrases).

  Returns:
      A dictionary containing extracted features.
  """
  keywords = ["login issue", "payment error", "security"]  # Define your keywords
  urgency_words = ["down", "critical", "urgent"]

  # Keyword matching
  keyword_features = {keyword: int(keyword in text) for keyword in keywords}

  # Sentiment analysis (using a pre-trained model)
  # Load your pre-trained sentiment analysis model here (e.g., TF Hub)
  sentiment = sentiment_model(tf.constant([text]))  # Replace with your model call

  # Urgency detection
  urgency_features = {"urgency": int(any(word in text for word in urgency_words))}

  # Combine features
  features = {**keyword_features, **sentiment, **urgency_features}
  return features


In [ ]:
def categorize(features):
  """
  This function categorizes tickets based on the extracted features using a model.

  Args:
      features: A dictionary containing extracted features.

  Returns:
      The predicted category label (string).
  """
  # Load your trained classification model here (e.g., multinomial logistic regression)
  model = tf.keras.Sequential([
      # Layers for your classification model based on features
  ])
  model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

  # Convert features to model input format (e.g., one-hot encoding)
  features = tf.convert_to_tensor(features)
  predictions = model.predict(tf.expand_dims(features, axis=0))
  category = category_list[tf.argmax(predictions[0])]  # Replace with your category list

  return category


In [ ]:
def prioritize(features):
  """
  This function assigns a priority score based on features.

  Args:
      features: A dictionary containing extracted features.

  Returns:
      A priority score (integer).
  """
  # Define weightage for different features (adjust as needed)
  weights = {"urgency": 3, "negative_sentiment": 2, "keyword_match": 1}

  # Calculate priority score (consider sentiment score and weightage)
  priority_score = 0
  if features["sentiment"] < 0.5:  # Adjust threshold based on sentiment model output
    priority_score += weights["negative_sentiment"]
  priority_score += features["urgency"] * weights["urgency"]
  for keyword, value in features.items():
